# Simulating aliquot sequencing for covariance matrix

Outside of this notebook I simulated sequencing of 192 aliquots of 20 gametes apiece, with 100 markers and 3,840,000 reads spread across those markers.

Here, I will revisit the recombination map and show how a **covariance matrix** contains the signal of genome-wide recombination variation.

### imports

In [1]:
import scipy.stats as st
import numpy as np
import scipy.integrate as integrate
import toyplot
import h5py

from tqdm.notebook import tqdm

import poolparty

## 1. Show the recombination map

### make a cool-shaped recombination map with all values greater than 0.

In [2]:
# now define scaling by that previous number:
scalar = 1 / integrate.quad(lambda x: (1+1*np.cos(21*x+np.sin(60*x))), 0, 1)[0] # one over previous line

# now look at new result (should equal 1!)
integrate.quad(lambda x: (1+1*np.cos(21*x+np.sin(60*x))) * scalar, 0, 1)[0]

1.0

In [3]:
toyplot.scatterplot(np.linspace(0,1,1000), 
                   (1+1*np.cos(21*np.linspace(0,1,num=1000)+np.sin(60*np.linspace(0,1,num=1000))))*scalar, # cool equation here
                   height=300,
                   width=500);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="300.0px" viewBox="0 0 500.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tafecdd3666f349418209000486ea8598"> <g style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:1.0;stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0" class="toyplot-Datum" transform="translate(403.95395

# 2. Simulate!

### This has already been done, with `poolparty`. 

# 3. Load simulated data.

In [72]:
haps = h5py.File('/pinky/patrick/poolparty_sims/sims/20gpa_192nali_100loci_3840e3reads/haplotypes.hdf5','r')

# 4. Examine distribution of markers.

### Where are the markers?

In [73]:
np.array(haps['loci_locs'])

array([0.01713903, 0.01939749, 0.03608618, 0.04482447, 0.05775471,
       0.05786713, 0.07590005, 0.08368368, 0.09580599, 0.10956318,
       0.11469308, 0.14549169, 0.14693218, 0.15594299, 0.16176115,
       0.1753468 , 0.17840618, 0.19012154, 0.19409192, 0.19450069,
       0.21793627, 0.2184831 , 0.24468038, 0.2523716 , 0.26221637,
       0.27584935, 0.28674107, 0.29964363, 0.32011889, 0.32895052,
       0.34597573, 0.37708686, 0.38280757, 0.38854184, 0.39250975,
       0.39839582, 0.40865566, 0.41356142, 0.43724481, 0.44045411,
       0.45562362, 0.46724957, 0.46746545, 0.48273026, 0.48536054,
       0.49024082, 0.49425538, 0.49434697, 0.4967879 , 0.49750747,
       0.51126486, 0.51980264, 0.53949535, 0.54851223, 0.57439135,
       0.59464173, 0.59874063, 0.60096924, 0.6056355 , 0.61057238,
       0.62353998, 0.67581999, 0.69065215, 0.69994757, 0.70049625,
       0.70289751, 0.70894795, 0.72931759, 0.73379558, 0.73901322,
       0.7412148 , 0.7464459 , 0.74745078, 0.75745521, 0.76035

### Look at the distribution...

In [74]:
toyplot.scatterplot(np.array(haps['loci_locs']),np.repeat(0,len(np.array(haps['loci_locs']))),height=300,width=1000);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="300.0px" viewBox="0 0 1000.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t4f9b46ca8d454a0ba9833878e7b58725"> 0.0 0.5 1.0 -0.5 0.0 0.5

# 5. Make a covariance matrix showing how the 100 markers covary in haplotype proportions across the 192 aliquots:

In [111]:
# make covariance matrix
cov_mat = np.zeros((100,192))
for row in tqdm(range(100)):
    # what what is locus Alpha, how many hap1s do we observe there?
    locusnumAlpha = row # locus location (out of 100)
    alphavals = np.zeros((192))
    for i in range(192):
        obs = haps[str(i)][str(locusnumAlpha)]
        alphavals[i] = np.sum(obs) / len(obs)

    #c = np.cov(alphavals,betavals)

    cov_mat[row,:] = alphavals

In [112]:
# 100 by 100 -- because we have 100 markers
np.cov(cov_mat).shape

(100, 100)

In [115]:
# plotting markers and recombination map on top, covariance matrix below:
canvas = toyplot.Canvas(width=800, height=300)
axes = canvas.cartesian(label = "Recombination map and loci locations")
mark = axes.scatterplot(np.linspace(0,1,1000), 
                   (1+1*np.cos(21*np.linspace(0,1,num=1000)+np.sin(60*np.linspace(0,1,num=1000))))*scalar)
mark = axes.scatterplot(np.array(haps['loci_locs']),np.repeat(1,len(np.array(haps['loci_locs']))))
toyplot.matrix(np.cov(cov_mat), tshow=False, lshow=False, label="Covariance matrix",width=800,height=800);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="800.0px" height="300.0px" viewBox="0 0 800.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tfbb093bec88649afaa15657913e955ae"> <g style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:1.0;stroke:rgb(40%,76.1%,64.7%);strok

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="800.0px" height="800.0px" viewBox="0 0 800.0 800.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tdd95406787be4c9cb4e9c3c03974a49e"> Covariance matrix 0.012959 0.011658 0.010719 0.010935 0.010469 0.010664 0.010085 0.009627 0.009505 0.009260 0.009797 0.009247 0.009011 0.010043 0.009682 0.009392 0.009672 0.009568 0.009508 0.009455 0.008883 0.008900 0.006969 0.006054 0.005952 0.004971 0.005309 0.004614 0.004052 0.003855 0.004041 0.002956 0.002745 0.002962 0.001890 0.002065 0.002478 0.001961 0.001876 0.001652 0.001560 0.001215 0.001257 0.000960 0.000778 0.000981 0.001348 0.001113 0.001113 0.001010 0.001224 0.001645 0.001163 0.000774 0.000267 -0.000497 -0.000977 -0.000920 -0.001266 -0.001008 -0.001141 -0.002543 -0.002942 -0.003839 -0.002576 -0.002770 -0.003106 -0.003767 -0.003949 -0.003393 -0.003842 -0.004282 -0.003887 -0.003952 -0.003544 -0.004265 -0.004514 -0.004387 -0.003943 -0.004632 -0.004652 -0.004534 -0.004529 -0.003860 -0.004657 -0.004479 -0.004051 -0.004405 -0.005367 -0.005549 -0.006230 -0.007057 -0.006973 -0.006917 -0.007416 -0.007076 -0.007288 -0.006894 -0.007072 -0.006867 0.011658 0.012892 0.010659 0.010902 0.010566 0.010890 0.009910 0.009637 0.009744 0.009270 0.009699 0.009154 0.009130 0.009836 0.009870 0.009479 0.009610 0.009677 0.009565 0.009652 0.008780 0.009137 0.006677 0.005971 0.005729 0.004933 0.005370 0.004837 0.003997 0.003654 0.004006 0.002834 0.002465 0.003265 0.001758 0.001831 0.002200 0.002012 0.001760 0.001459 0.001516 0.001001 0.001207 0.000939 0.000801 0.001025 0.001294 0.001095 0.001003 0.000789 0.001234 0.001470 0.001190 0.000639 0.000106 -0.000628 -0.001449 -0.001245 -0.001718 -0.001101 -0.001362 -0.002852 -0.003090 -0.003908 -0.002558 -0.002758 -0.003080 -0.003762 -0.003865 -0.003577 -0.003678 -0.004214 -0.003799 -0.003837 -0.003554 -0.004248 -0.004332 -0.004332 -0.003754 -0.004376 -0.004492 -0.004449 -0.004387 -0.003961 -0.004612 -0.004402 -0.004063 -0.004306 -0.005411 -0.005683 -0.006118 -0.006724 -0.006902 -0.006691 -0.007356 -0.007178 -0.007347 -0.007059 -0.006855 -0.006861 0.010719 0.010659 0.011534 0.010662 0.010355 0.010574 0.009864 0.009661 0.009413 0.009250 0.009520 0.009057 0.008941 0.009564 0.009609 0.009167 0.009065 0.009304 0.009308 0.009492 0.008566 0.008656 0.006943 0.005940 0.005760 0.004843 0.005176 0.005313 0.004232 0.003647 0.003908 0.002722 0.002300 0.002575 0.001534 0.001663 0.002162 0.001712 0.001346 0.001260 0.001380 0.000700 0.001149 0.000731 0.000352 0.000660 0.000905 0.000937 0.000777 0.000661 0.000722 0.001354 0.000881 0.000733 -0.000322 -0.000626 -0.001343 -0.001273 -0.001834 -0.001364 -0.001534 -0.002655 -0.003095 -0.003830 -0.002683 -0.003048 -0.003027 -0.003690 -0.003895 -0.003649 -0.003846 -0.004393 -0.003952 -0.003699 -0.003420 -0.004198 -0.004454 -0.004436 -0.003846 -0.004239 -0.004709 -0.004437 -0.004268 -0.003862 -0.004407 -0.004511 -0.004093 -0.004468 -0.005489 -0.005633 -0.005924 -0.006351 -0.006541 -0.006200 -0.006777 -0.006496 -0.006705 -0.006308 -0.006368 -0.006298 0.010935 0.010902 0.010662 0.012914 0.011050 0.011698 0.010445 0.009936 0.010238 0.010070 0.010196 0.010055 0.009759 0.010242 0.010220 0.009987 0.009798 0.010145 0.010002 0.010287 0.009647 0.009661 0.007484 0.006369 0.006354 0.005608 0.006017 0.005955 0.005300 0.004447 0.004807 0.003986 0.003527 0.003708 0.002311 0.002738 0.003321 0.003121 0.002568 0.002471 0.002753 0.002275 0.002191 0.002053 0.001704 0.001973 0.002154 0.002097 0.001939 0.002067 0.002061 0.002695 0.001965 0.001684 0.000332 -0.000237 -0.000766 -0.000616 -0.001297 -0.000910 -0.001212 -0.002339 -0.002772 -0.003112 -0.002007 -0.0